## Inventary


#### admin acount required

In [3]:
from arcgis.gis import GIS
gis = GIS("http://www.arcgis.com/","rmartin_esri_colombia","******")
import pandas as pd
from datetime import datetime
from time import sleep

In [4]:
gis.admin.credits.credits

-2661.1194

In [5]:
gis.users.search()

[<User username:agomez_esri_colombia>,
 <User username:Alexandra_Gomez_esri_colombia>,
 <User username:Andrea_Carolina_Castillo_esri_colombia>,
 <User username:cgonzalez_esri_colombia>,
 <User username:CommunityMaps_EsriColombia>,
 <User username:CuratorCO_Esri_Colombia>,
 <User username:dfonseca_esri_colombia>,
 <User username:ecruz_esri_colombia>,
 <User username:Eduardo_Cruz_esri_colombia>,
 <User username:esri_pruebas>,
 <User username:jvillegas_esri_colombia>,
 <User username:lgonzalez_esri_colombia>,
 <User username:Manuel_Francisco_Lemos_esri_colombia>,
 <User username:marketplace_esri_colombia>,
 <User username:rmartin_esri_colombia>,
 <User username:Robert_Andres_Pulido_esri_colombia>,
 <User username:rpulido_esri_colombia>]

#### Inventary

In [7]:
columns = ["Folder","Propietario", "Título", "Nombre", "Tipo", "Número de vistas", 
           "Tamaño (mb)", "Fecha creación", "Fecha modificación", "Item ID","Acceso","Url"]
def getInventaryItems(user):    
    datos = []
    foldersName = [None]+[folder["title"] for folder in user.folders]
    for folder in foldersName:
        items = user.items(folder)
        if folder is None:
            folder = "Root"
        for item in items:
            datos.append([folder, 
                          item.owner, 
                          item.title, 
                          item.name, 
                          item.type, 
                          item.numViews, 
                          item.size/1048576 , 
                          datetime.utcfromtimestamp(item.created/1000).strftime('%Y-%m-%d %H:%M:%S'),
                          datetime.utcfromtimestamp(item.modified/1000).strftime('%Y-%m-%d %H:%M:%S'),
                          item.id,
                          item.access,
                          item.url])
    return datos   

**Note: Executing the following cell may take a few minutes.**

In [8]:
df_i = pd.DataFrame()

##limit the users
for user in gis.users.search():
    datos = getInventaryItems(user)
    df = pd.DataFrame(datos, columns=columns)
    df_i=pd.concat([df_i,df])
df_i['GB']=df_i['Tamaño (mb)']/1024

In [16]:
df_i.to_csv('inventary.csv',encoding='utf8',sep=';',index=False)

In [10]:
print("Total almacenamiento de {} Mb == {} Gb".format(df_i['Tamaño (mb)'].sum(),df_i['GB'].sum()))

Total almacenamiento de 3989.644578933716 Mb == 3.896137284114957 Gb


In [11]:
# para mostrar muchos registros
pd.set_option('display.max_rows', 500)

In [12]:
len(df_i['Propietario'].unique())

13

### Calculate credits : https://doc.arcgis.com/en/arcgis-online/administer/credits.htm NOT already

# Por tipo

In [14]:
# weight by type
types=df_i.groupby(['Tipo']).sum().reset_index()
w = {t:float(types.loc[(types['Tipo'] == t), 'Tamaño (mb)']) for (t) in list(types.Tipo.unique())}
types

,Tipo,Tamaño (mb),GB
0,CSV,13.420790,1.310624e-02
1,Code Attachment,0.000000,0.000000e+00
2,Dashboard,0.219367,2.142256e-04
3,Document Link,0.000119,1.164153e-07
4,Feature Collection,0.535104,5.225623e-04
5,Feature Service,2058.887380,2.010632e+00
6,File Geodatabase,64.920511,6.339894e-02
7,Form,0.448763,4.382450e-04
8,Geocoding Service,0.000128,1.247972e-07
9,Hub Page,0.021961,2.144650e-05


### Per type

In [46]:
#by owner
owner=df_i.groupby(['Propietario']).sum().sort_values('Tamaño (mb)',ascending=False)
owner

,Número de vistas,Tamaño (mb),GB
Propietario,,,
PublicadorSIG,2097243,6603.892309,6.449114
mtoro_geek,30062,4015.362970,3.921253
mpenagos_geek,21647,1762.048035,1.720750
Spedraza_geek,51662,1490.799701,1.455859
rcarvajal_geek,49055,1052.248143,1.027586
cquilcue_geek,50903,860.791024,0.840616
rmartin_geek,83327,618.356558,0.603864
kvasquez_geek,4504,539.447357,0.526804
nmedina_geek,15564,330.227410,0.322488


In [45]:
#by owner and type
owner_type=df_i.groupby(['Propietario','Tipo']).sum().sort_values('Tamaño (mb)',ascending=False)
owner_type

,,Número de vistas,Tamaño (mb),GB
Propietario,Tipo,,,
PublicadorSIG,Feature Service,629994,3459.934459,3.378842e+00
mtoro_geek,Service Definition,86,2159.388775,2.108778e+00
mpenagos_geek,Feature Service,11750,1425.281384,1.391876e+00
Spedraza_geek,Feature Service,35108,1185.222442,1.157444e+00
PublicadorSIG,Service Definition,1307,1146.363842,1.119496e+00
mtoro_geek,Map Service,527,969.414856,9.466942e-01
PublicadorSIG,Web Mapping Application,506721,880.543422,8.599057e-01
rcarvajal_geek,Feature Service,46824,765.546958,7.476045e-01
mtoro_geek,Feature Service,10551,694.968750,6.786804e-01


In [60]:
#by owner and folder
owner_folder=df_i.groupby(['Propietario','Folder']).sum().sort_values('Tamaño (mb)',ascending=False)
owner_folder.reset_index()

,Propietario,Folder,Tamaño (mb),GB
0,mtoro_geek,Cambio cobertura de la tierra,2886.346983,2.818698
1,PublicadorSIG,Survey-ESRIFTC2020,1212.633162,1.184212
2,PublicadorSIG,Antropoceno,1153.658388,1.126620
3,mpenagos_geek,Australia,831.016688,0.811540
4,rcarvajal_geek,Pablo_Leyva,809.120505,0.790157
...,...,...,...,...
467,drodriguez_geek,Root,0.000000,0.000000
468,PublicadorSIG,App Eventos Nueva 2019,0.000000,0.000000
469,drodriguez_geek,wab,0.000000,0.000000
470,PublicadorSIG,Aplicaciones migradas - Curador,0.000000,0.000000


In [50]:
owner_item=df_i.groupby(['Propietario','Título','Tipo','Item ID','Fecha creación','Fecha modificación']).sum().sort_values('Número de vistas',ascending=False).reset_index()
owner_item

,Propietario,Título,Tipo,Item ID,Fecha creación,Fecha modificación,Número de vistas,Tamaño (mb),GB
0,PublicadorSIG,partners_Esri_2016,Feature Service,0cc5edf5c38e47ca914fa46f4189f40e,2016-07-21 18:35:30,2018-12-13 16:45:56,135618,1.593750,1.556396e-03
1,PublicadorSIG,ExterioresHyatt_CCU2018,Feature Service,36da55a08a95498abaff80319582662c,2018-09-05 17:49:52,2018-09-05 18:07:39,108285,0.140625,1.373291e-04
2,PublicadorSIG,1810xcapas,Feature Service,944e8368c3fa43fbaa4dfaed1fd4e213,2017-07-12 20:24:53,2017-07-12 20:25:21,48773,1.242188,1.213074e-03
3,PublicadorSIG,Así se impusieron Duque y Petro en las consult...,Web Mapping Application,c31da8a3a6944546931b0bbea59248c8,2018-03-08 02:24:38,2018-03-13 01:56:13,41356,24.051664,2.348795e-02
4,rmartin_geek,Boyacá Compleja,Hub Site Application,ee808c223ce94f6d8dbadb5fe395eb51,2019-01-31 17:14:37,2020-05-07 19:08:13,39366,63.499037,6.201078e-02
...,...,...,...,...,...,...,...,...,...
5403,aramirez_geek,BA Web App Smart Map Search Data,Web Mapping Application,783339f786c8403c8c6526614c496a69,2020-08-14 18:52:08,2020-08-14 18:52:08,0,0.000010,1.024455e-08
5404,aramirez_geek,BA Web App Comparison Reports Variables,Web Mapping Application,35804400a346461882e69a073d7afa6a,2020-08-14 18:52:08,2020-08-14 18:52:08,0,0.000010,1.024455e-08
5405,aramirez_geek,BA Web App Comparison Reports Data,Web Mapping Application,740fead5cc8549fcaa66a69e147ac86e,2020-08-14 18:52:08,2020-08-14 18:52:08,0,0.000010,1.024455e-08
5406,aramirez_geek,,StoryMap,e72fbdabc4434488a329fb47cf5901f4,2020-03-19 00:22:14,2020-03-19 00:23:24,0,0.000635,6.202608e-07


In [42]:
#by owner
df_i.query("Propietario == 'mtoro_geek'").sort_values('Tamaño (mb)',ascending=False)

,Folder,Propietario,Título,Nombre,Tipo,Número de vistas,Tamaño (mb),Fecha creación,Fecha modificación,Item ID,Acceso,GB
62,Cambio cobertura de la tierra,mtoro_geek,AREAS_PROTEGIDAS_MUNDO_WFL1,AREAS_PROTEGIDAS_MUNDO_WFL1.sd,Service Definition,3,1139.991931,2019-05-30 22:40:16,2019-05-30 22:40:19,209d92b21498443f9bd4f1d5ab2ff279,public,1.113273e+00
63,Cambio cobertura de la tierra,mtoro_geek,SUPERFICIES_AGUA_WFL1,SUPERFICIES_AGUA_WFL1.sd,Service Definition,2,310.638158,2019-05-31 21:19:30,2019-05-31 21:19:31,10d72421d2494577a6eca4b375a90a58,public,3.033576e-01
66,Cambio cobertura de la tierra,mtoro_geek,MAPA CLC COBERTURAS_WTL1,MAPA_CLC_COBERTURAS_WTL1.sd,Service Definition,0,277.415854,2019-07-15 20:54:14,2019-07-15 20:54:15,f7075964c1d54b63910774631a522fde,private,2.709139e-01
67,Cambio cobertura de la tierra,mtoro_geek,MAPA CLC COBERTURAS_WTL1,MAPA_CLC_COBERTURAS_WTL1,Map Service,18,277.415854,2019-07-15 20:54:19,2019-07-15 20:54:20,6a617ee69a05497c8402ade9b5616584,public,2.709139e-01
80,Cambio cobertura de la tierra,mtoro_geek,clc 2008_WTL1,clc_2008_WTL1,Map Service,108,214.540657,2019-07-16 22:56:49,2019-07-16 23:01:47,b037311515314e00b1a4145d455ddf8b,public,2.095124e-01
73,Cambio cobertura de la tierra,mtoro_geek,CLC 2013_WTL1,CLC_2013_WTL1,Map Service,121,213.509151,2019-07-16 21:31:50,2019-07-16 21:42:06,9942ea48674c4c038c884870f1e2be24,public,2.085050e-01
71,Cambio cobertura de la tierra,mtoro_geek,CLC 2003_WTL1,CLC_2003_WTL1,Map Service,139,177.467700,2019-07-15 21:34:45,2019-07-15 21:38:20,6f63fafd06a54b959fc623ffabd5630c,public,1.733083e-01
140,Mapa socioecosistemas,mtoro_geek,MAPA SOCIOECOSISTEMAS CAPAS_WFL1,MAPA_SOCIOECOSISTEMAS_CAPAS_WFL1,Feature Service,500,168.164062,2019-07-17 19:23:55,2019-07-17 19:27:34,10c787a4004143ed84782e8e9807c445,public,1.642227e-01
34,Root,mtoro_geek,Mapas que inspiran el futuro.,None,Web Mapping Application,198,144.029045,2019-06-07 19:20:43,2019-06-26 22:16:40,d773f490fa8747308e348f25cccbf566,private,1.406534e-01
72,Cambio cobertura de la tierra,mtoro_geek,CLC 2013_WTL1,CLC_2013_WTL1.sd,Service Definition,1,126.443832,2019-07-16 21:31:48,2019-07-16 21:31:49,bcc89bb62e2a4b27af79fb5b96e3b9f1,public,1.234803e-01


In [41]:
#  by type
df_i[df_i['Tipo']=='Service Definition'].sort_values('Tamaño (mb)',ascending=False)

,Folder,Propietario,Título,Nombre,Tipo,Número de vistas,Tamaño (mb),Fecha creación,Fecha modificación,Item ID,Acceso,GB
62,Cambio cobertura de la tierra,mtoro_geek,AREAS_PROTEGIDAS_MUNDO_WFL1,AREAS_PROTEGIDAS_MUNDO_WFL1.sd,Service Definition,3,1139.991931,2019-05-30 22:40:16,2019-05-30 22:40:19,209d92b21498443f9bd4f1d5ab2ff279,public,1.113273
63,Cambio cobertura de la tierra,mtoro_geek,SUPERFICIES_AGUA_WFL1,SUPERFICIES_AGUA_WFL1.sd,Service Definition,2,310.638158,2019-05-31 21:19:30,2019-05-31 21:19:31,10d72421d2494577a6eca4b375a90a58,public,0.303358
66,Cambio cobertura de la tierra,mtoro_geek,MAPA CLC COBERTURAS_WTL1,MAPA_CLC_COBERTURAS_WTL1.sd,Service Definition,0,277.415854,2019-07-15 20:54:14,2019-07-15 20:54:15,f7075964c1d54b63910774631a522fde,private,0.270914
289,Antropoceno,PublicadorSIG,ÁREAS PROTEGIDAS MUNDIALES,ÁREAS_PROTEGIDAS_MUNDIALES.sd,Service Definition,1,222.800217,2019-06-18 17:03:31,2020-01-22 19:25:00,6e51166426ba430a903272d7f7f615c5,private,0.217578
72,Cambio cobertura de la tierra,mtoro_geek,CLC 2013_WTL1,CLC_2013_WTL1.sd,Service Definition,1,126.443832,2019-07-16 21:31:48,2019-07-16 21:31:49,bcc89bb62e2a4b27af79fb5b96e3b9f1,public,0.123480
...,...,...,...,...,...,...,...,...,...,...,...,...
1645,"Fundación Alma, Somos Páramo",PublicadorSIG,Talleres,Talleres.sd,Service Definition,1,0.037849,2018-11-16 15:41:59,2018-11-16 15:45:40,a9fbcfdef12f486eb9c447eb94ba6a77,private,0.000037
1641,"Fundación Alma, Somos Páramo",PublicadorSIG,Foros,Foros.sd,Service Definition,1,0.037821,2018-11-16 15:36:52,2018-11-16 15:45:40,bdf3311c6db44be1adcf9b9674989fb2,private,0.000037
34,Root,kvasquez_geek,prueba gestion,prueba_gestion.sd,Service Definition,1,0.036471,2019-02-20 21:42:29,2019-02-20 21:42:30,899b46db4896418f89045e2b8d400e80,private,0.000036
1629,"Fundación Alma, Proyectos",PublicadorSIG,Puntos2,Puntos2.sd,Service Definition,1,0.036262,2018-12-05 19:54:09,2019-09-09 16:19:23,3febe845a123426f877be0d4384902b0,private,0.000035


# vistas

In [51]:
df_i.groupby(['Título']).sum().reset_index()

,Título,Número de vistas,Tamaño (mb),GB
0,,0,0.001002,9.788200e-07
1,Denver Map Gallery,2,1.087397,1.061911e-03
2,Mid-Century,2,1.016488,9.926641e-04
3,$A$1:$AW$485 por Family,7,0.242188,2.365112e-04
4,.,191,4.226326,4.127271e-03
...,...,...,...,...
4164,Índice de humedad mundial,275,0.003956,3.863126e-06
4165,Últimos movimientos teluricos,8588,0.003297,3.219582e-06
4166,áreas protegidas,79,0.009228,9.011477e-06
4167,índice_de_humedad,92,2.034296,1.986617e-03


### save

In [65]:
# df_i.to_csv("inventary.csv",encoding='utf-8-sig')
df_i.groupby(['Tipo']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/type.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Tipo']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_type.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Folder']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_folder.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Título','Item ID','Tipo','Fecha creación','Fecha modificación']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_item.csv",encoding='utf-8-sig',sep=';',decimal=',')
#download in Files tab; and click on file

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('inventary.csv',sep=';')
df.dtypes

Folder                 object
Propietario            object
Título                 object
Nombre                 object
Tipo                   object
Número de vistas        int64
Tamaño (mb)           float64
Fecha creación         object
Fecha modificación     object
Item ID                object
Acceso                 object
Url                    object
GB                    float64
dtype: object

In [10]:
df.to_parquet('inventary.parquet.gzip',
              compression='gzip')  

In [9]:
!pip install pyarrow

     |████████████████████████████████| 17.7 MB 139 kB/s eta 0:00:01    |█▍                              | 768 kB 6.5 MB/s eta 0:00:03
